In [1]:
# import transforms and datasets in TorchVision
import torchvision.transforms as T
import torchvision.datasets as datasets

# load or download MNIST datasets
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=T.ToTensor())
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=T.ToTensor())

In [2]:
# import DataLoader in PyTorch
from torch.utils.data import DataLoader

# create data loaders to feed data into our model
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [3]:
# check the data from the data loader
for X, y in train_dataloader:
    print(f'Shape of X [batch, channel, height, width]: {X.shape} ({X.dtype})') 
    print(f'Shape of y: {y.shape} ({y.dtype})')
    break

Shape of X [batch, channel, height, width]: torch.Size([64, 1, 28, 28]) (torch.float32)
Shape of y: torch.Size([64]) (torch.int64)


In [4]:
# import nn in PyTorch
import torch
import torch.nn as nn

# define MLP network with one hidden layer (original version)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten() # flatten the input tensor as a 1D vector ((28, 28) -> (784))
        self.input_layer = nn.Linear(28*28, 512)
        self.hidden_layer = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        h = self.relu(self.input_layer(x))
        h = self.relu(self.hidden_layer(h))
        y = self.output_layer(h)
        return y

In [5]:
# import cuda in PyTorch
import torch.cuda as cuda

# check device for training
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


In [6]:
# get the MLP model and send it to the device
myMLP = MLP().to(device)
print(myMLP)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (input_layer): Linear(in_features=784, out_features=512, bias=True)
  (hidden_layer): Linear(in_features=512, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
)


In [9]:
# define a loss function and an optimizer
loss_fn = nn.CrossEntropyLoss(reduction='mean') # cross entropy loss function for classification
optimizer = torch.optim.SGD(myMLP.parameters(), lr=5e-3) # stochastic gradient descent with learning rate of 0.005

print(loss_fn)
print(optimizer)

CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.005
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [8]:
# train the MLP model
epochs = 5
ndata = len(train_dataloader.dataset) 
print('training starts!')

for e in range(epochs):
    print(f'\nepoch {e+1}\n------------------------------')
    myMLP.train() # train mode
    
    for b, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device) # input and target to device(gpu)

        prediction = myMLP(X) # forward pass
        train_loss = loss_fn(prediction, y) # calculate the loss 

        optimizer.zero_grad() # clear gradients
        train_loss.backward() # backpropagation
        optimizer.step() # update the parameters

        if b % 100 == 0: # track the training
            train_loss, current = train_loss.item(), b * len(X) 
            print(f'loss: {train_loss:>7f}  [{current:>5d}/{ndata:>5d}]')
print('\ntraining is finished!')

training starts!

epoch 1
------------------------------
loss: 2.291039  [    0/60000]
loss: 2.275304  [ 6400/60000]
loss: 2.275133  [12800/60000]
loss: 2.211379  [19200/60000]
loss: 2.194322  [25600/60000]
loss: 2.147560  [32000/60000]
loss: 2.054236  [38400/60000]
loss: 2.061951  [44800/60000]
loss: 1.893292  [51200/60000]
loss: 1.730148  [57600/60000]

epoch 2
------------------------------
loss: 1.700732  [    0/60000]
loss: 1.491445  [ 6400/60000]
loss: 1.452919  [12800/60000]
loss: 1.139206  [19200/60000]
loss: 1.039339  [25600/60000]
loss: 0.933566  [32000/60000]
loss: 0.805866  [38400/60000]
loss: 0.913249  [44800/60000]
loss: 0.800716  [51200/60000]
loss: 0.713659  [57600/60000]

epoch 3
------------------------------
loss: 0.764488  [    0/60000]
loss: 0.603892  [ 6400/60000]
loss: 0.634781  [12800/60000]
loss: 0.583562  [19200/60000]
loss: 0.528263  [25600/60000]
loss: 0.513650  [32000/60000]
loss: 0.427318  [38400/60000]
loss: 0.598129  [44800/60000]
loss: 0.557169  [51200/

In [10]:
# test the MLP model
ndata = len(test_dataloader.dataset)
nbatch = len(test_dataloader)
myMLP.eval() # test mode
test_loss, correct = 0, 0 # initialize

with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        prediction = myMLP(X)
        test_loss += loss_fn(prediction, y).item() # add up the loss
        test_ += (prediction.argmax(1) == y).type(torch.float).sum().item() # add up the correct predictions
test_loss /= nbatch # average the loss
correct /= ndata # accuracy for all data
print(f"test error\n-> accuracy: {(100*correct):>0.1f}%, average loss: {test_loss:>8f}")

test error
-> accuracy: 89.9%, average loss: 0.354588
